In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample

# --- 1. CẤU HÌNH ---
DATA_PATH = '/kaggle/input/MABe-mouse-behavior-detection/'
NUM_VIDEOS_TO_TRAIN = 20 # Số lượng video dùng để train (Càng nhiều càng tốt)

# Đọc metadata
try:
    df_train_meta = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
except FileNotFoundError:
    print("⚠️ Lỗi: Không tìm thấy file train.csv")

# --- 2. HÀM TẠO ĐẶC TRƯNG (FEATURE ENGINEERING) ---
def calculate_features_with_memory(df):
    # a. Tính Vật lý (Distance & Velocity)
    try:
        dx = df['mouse1_body_center_x'] - df['mouse2_body_center_x']
        dy = df['mouse1_body_center_y'] - df['mouse2_body_center_y']
        df['distance'] = np.sqrt(dx**2 + dy**2)
    except KeyError:
        df['distance'] = 0 
        
    vx = df['mouse1_body_center_x'].diff().fillna(0)
    vy = df['mouse1_body_center_y'].diff().fillna(0)
    df['velocity_m1'] = np.sqrt(vx**2 + vy**2)
    
    try:
        vx2 = df['mouse2_body_center_x'].diff().fillna(0)
        vy2 = df['mouse2_body_center_y'].diff().fillna(0)
        df['velocity_m2'] = np.sqrt(vx2**2 + vy2**2)
    except KeyError:
        df['velocity_m2'] = 0
        
    # b. Tạo Ký ức (Rolling Window - 10 frames)
    w = 10
    df['dist_mean_10'] = df['distance'].rolling(window=w).mean().fillna(0)
    df['dist_std_10'] = df['distance'].rolling(window=w).std().fillna(0)
    df['vel1_mean_10'] = df['velocity_m1'].rolling(window=w).mean().fillna(0)
    df['vel2_mean_10'] = df['velocity_m2'].rolling(window=w).mean().fillna(0)
    
    return df

# --- 3. HÀM LOAD & LỌC DỮ LIỆU (CHỈ LẤY CẶP 1-2 ĐỂ TRAIN) ---
def get_train_data(idx):
    row = df_train_meta.iloc[idx]
    lab_id, video_id = row['lab_id'], row['video_id']
    pix_per_cm = row['pix_per_cm_approx'] if row['pix_per_cm_approx'] > 0 else 1.0
    
    # Load Tracking
    t_path = os.path.join(DATA_PATH, 'train_tracking', lab_id, f'{video_id}.parquet')
    a_path = os.path.join(DATA_PATH, 'train_annotation', lab_id, f'{video_id}.parquet')
    
    try:
        df_track = pd.read_parquet(t_path)
    except FileNotFoundError: return None

    # Pivot & Chuẩn hóa CM
    px = df_track.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='x')
    px.columns = [f"mouse{m}_{bp}_x" for m, bp in px.columns]
    py = df_track.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='y')
    py.columns = [f"mouse{m}_{bp}_y" for m, bp in py.columns]
    df_wide = pd.concat([px, py], axis=1).sort_index(axis=1)
    df_wide = df_wide / pix_per_cm 

    # Load Annotation & Lọc chỉ lấy tương tác cặp 1-2
    try:
        df_annot = pd.read_parquet(a_path)
        df_wide['label'] = 'other'
        # Chỉ lấy dòng có agent=1, target=2 HOẶC agent=2, target=1
        mask = ((df_annot['agent_id'] == 1) & (df_annot['target_id'] == 2)) | \
               ((df_annot['agent_id'] == 2) & (df_annot['target_id'] == 1))
        pair_annot = df_annot[mask]
        
        for _, r in pair_annot.iterrows():
            if r['stop_frame'] <= df_wide.index.max():
                df_wide.loc[r['start_frame']:r['stop_frame'], 'label'] = r['action']
    except:
        return None # Bỏ qua video lỗi nhãn

    return df_wide.fillna(0)

# --- 4. PIPELINE CHÍNH: GỘP DATA -> CÂN BẰNG -> TRAIN ---
features = ['distance', 'velocity_m1', 'velocity_m2', 
            'dist_mean_10', 'dist_std_10', 'vel1_mean_10', 'vel2_mean_10']

# A. Gộp dữ liệu nhiều video
all_data = []
print(f"⏳ Đang xử lý {NUM_VIDEOS_TO_TRAIN} video...")
for i in tqdm(range(NUM_VIDEOS_TO_TRAIN)):
    df = get_train_data(i)
    if df is not None and len(df) > 0:
        df = calculate_features_with_memory(df)
        all_data.append(df[features + ['label']])

df_big_train = pd.concat(all_data, ignore_index=True)
print(f"✅ Kích thước tập Train thô: {df_big_train.shape}")

# B. Cân bằng dữ liệu (Undersampling an toàn)
print("⚖️ Đang cân bằng dữ liệu...")
others = df_big_train[df_big_train['label'] == 'other']
actions = df_big_train[df_big_train['label'] != 'other']

min_sample = min(len(others), len(actions))
df_bal = pd.concat([
    resample(others, replace=False, n_samples=min_sample, random_state=42),
    resample(actions, replace=False, n_samples=min_sample, random_state=42)
])
print(f"✅ Dữ liệu sau cân bằng: {df_bal.shape} (Mỗi phe {min_sample} dòng)")

# C. Train Model
print("🚀 Đang huấn luyện Random Forest...")
le = LabelEncoder()
y_train = le.fit_transform(df_bal['label'])
X_train = df_bal[features]

model_big = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
model_big.fit(X_train, y_train)
print("🎉 Đã huấn luyện xong model_big!")
print(f"Các nhãn đã học: {le.classes_}")

⏳ Đang xử lý 20 video...


  0%|          | 0/20 [00:00<?, ?it/s]

✅ Kích thước tập Train thô: (2512390, 8)
⚖️ Đang cân bằng dữ liệu...
✅ Dữ liệu sau cân bằng: (48634, 8) (Mỗi phe 24317 dòng)
🚀 Đang huấn luyện Random Forest...
🎉 Đã huấn luyện xong model_big!
Các nhãn đã học: ['approach' 'attack' 'avoid' 'chase' 'chaseattack' 'other' 'shepherd'
 'submit']


In [2]:
import pandas as pd
import numpy as np
import os
import gc
import lightgbm as lgb
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
import itertools

# --- 1. CẤU HÌNH ---
DATA_PATH = '/kaggle/input/MABe-mouse-behavior-detection/'
NUM_VIDEOS_TRAIN = 50 

# --- 2. HÀM FEATURE ENGINEERING AN TOÀN (FIX LỖI KEYERROR) ---
def calculate_features_safe(df):
    # Khởi tạo mặc định bằng 0.0 để tránh lỗi thiếu cột
    df['distance'] = 0.0
    df['velocity_m1'] = 0.0
    df['velocity_m2'] = 0.0
    
    # 1. Tính Khoảng cách (Chỉ khi có đủ cả 2 chuột)
    if 'mouse1_body_center_x' in df.columns and 'mouse2_body_center_x' in df.columns:
        dx = df['mouse1_body_center_x'] - df['mouse2_body_center_x']
        dy = df['mouse1_body_center_y'] - df['mouse2_body_center_y']
        df['distance'] = np.sqrt(dx**2 + dy**2)
    
    # 2. Tính Vận tốc M1
    if 'mouse1_body_center_x' in df.columns:
        vx1 = df['mouse1_body_center_x'].diff().fillna(0)
        vy1 = df['mouse1_body_center_y'].diff().fillna(0)
        df['velocity_m1'] = np.sqrt(vx1**2 + vy1**2)
        
    # 3. Tính Vận tốc M2
    if 'mouse2_body_center_x' in df.columns:
        vx2 = df['mouse2_body_center_x'].diff().fillna(0)
        vy2 = df['mouse2_body_center_y'].diff().fillna(0)
        df['velocity_m2'] = np.sqrt(vx2**2 + vy2**2)
        
    # 4. Tính Ký ức (Rolling Window)
    # Vì các cột trên đã được khởi tạo (dù là 0), nên đoạn này luôn an toàn
    w = 10
    df['dist_mean_10'] = df['distance'].rolling(window=w).mean().fillna(0)
    df['vel1_mean_10'] = df['velocity_m1'].rolling(window=w).mean().fillna(0)
    df['vel2_mean_10'] = df['velocity_m2'].rolling(window=w).mean().fillna(0)
    
    return df

features = ['distance', 'velocity_m1', 'velocity_m2', 'dist_mean_10', 'vel1_mean_10', 'vel2_mean_10']

# --- 3. HÀM LOAD DATA "ALL-PAIRS" (ĐÃ THÊM TRY-EXCEPT) ---
def load_train_data_all_pairs(meta_row):
    try:
        video_id = meta_row['video_id']
        lab_id = meta_row['lab_id']
        pix_per_cm = meta_row['pix_per_cm_approx'] if meta_row['pix_per_cm_approx'] > 0 else 1.0
        
        t_path = os.path.join(DATA_PATH, 'train_tracking', lab_id, f'{video_id}.parquet')
        a_path = os.path.join(DATA_PATH, 'train_annotation', lab_id, f'{video_id}.parquet')
        
        if not os.path.exists(t_path) or not os.path.exists(a_path): return None
        
        # Load & Pivot
        df_track = pd.read_parquet(t_path)
        px = df_track.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='x')
        px.columns = [f"mouse{m}_{bp}_x" for m, bp in px.columns]
        py = df_track.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='y')
        py.columns = [f"mouse{m}_{bp}_y" for m, bp in py.columns]
        df_wide = pd.concat([px, py], axis=1).sort_index(axis=1)
        
        # Interpolate & Normalize
        df_wide = df_wide.interpolate(limit=5).fillna(0)
        df_wide = df_wide / pix_per_cm 
        
        # Load Annotation
        df_annot = pd.read_parquet(a_path)
        
        mouse_ids = sorted(list(set([int(c.split('_')[0].replace('mouse', '')) for c in df_wide.columns if 'mouse' in c])))
        pairs_data = []
        
        for m1, m2 in itertools.combinations(mouse_ids, 2):
            cols1 = [c for c in df_wide.columns if f'mouse{m1}_' in c]
            cols2 = [c for c in df_wide.columns if f'mouse{m2}_' in c]
            if not cols1 or not cols2: continue
            
            df_pair = df_wide[cols1 + cols2].copy()
            rename_dict = {}
            for c in cols1: rename_dict[c] = c.replace(f'mouse{m1}_', 'mouse1_')
            for c in cols2: rename_dict[c] = c.replace(f'mouse{m2}_', 'mouse2_')
            df_pair.rename(columns=rename_dict, inplace=True)
            
            # Check xem sau khi rename có đủ cột body_center không, nếu không thì hàm safe sẽ xử lý
            
            df_pair['label'] = 'other'
            mask = ((df_annot['agent_id'] == m1) & (df_annot['target_id'] == m2)) | \
                   ((df_annot['agent_id'] == m2) & (df_annot['target_id'] == m1))
            
            pair_annot = df_annot[mask]
            for _, r in pair_annot.iterrows():
                if r['stop_frame'] <= df_pair.index.max():
                    df_pair.loc[r['start_frame']:r['stop_frame'], 'label'] = r['action']
                    
            pairs_data.append(df_pair)
            
        if pairs_data:
            return pd.concat(pairs_data, ignore_index=True)
    except Exception as e:
        # print(f"Skipping video {meta_row['video_id']} due to error: {e}")
        return None
        
    return None

# --- 4. CHUẨN BỊ DỮ LIỆU TRAIN ---
print("⏳ Đang tạo dữ liệu 'All-Pairs' từ 50 video (Safe Mode)...")
try:
    df_meta = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
except:
    df_meta = pd.DataFrame()

all_train_dfs = []
# Dùng tqdm để theo dõi tiến độ
for i in tqdm(range(min(NUM_VIDEOS_TRAIN, len(df_meta)))):
    df = load_train_data_all_pairs(df_meta.iloc[i])
    if df is not None:
        # Dùng hàm SAFE thay vì hàm thường
        df = calculate_features_safe(df) 
        all_train_dfs.append(df[features + ['label']])

if len(all_train_dfs) > 0:
    df_train_big = pd.concat(all_train_dfs, ignore_index=True)
    del all_train_dfs
    gc.collect()

    print(f"✅ Dữ liệu thô: {df_train_big.shape}")

    # --- 5. CÂN BẰNG DỮ LIỆU ---
    print("⚖️ Đang cân bằng dữ liệu...")
    others = df_train_big[df_train_big['label'] == 'other']
    actions = df_train_big[df_train_big['label'] != 'other']

    n_sample = min(len(others), len(actions))
    # Nếu video quá ít action, ta lấy toàn bộ action và một phần other
    if n_sample == 0: n_sample = 1000 # Fallback
    
    df_train_bal = pd.concat([
        resample(others, replace=False, n_samples=n_sample, random_state=42),
        actions # Lấy hết action (vì thường action ít hơn other)
    ])
    
    # Nếu action nhiều hơn other (hiếm), ta cũng có thể resample action. 
    # Nhưng code trên ưu tiên giữ lại toàn bộ hành vi hiếm.
    
    print(f"✅ Dữ liệu train cuối cùng: {df_train_bal.shape}")

    # Label Encoding
    le = LabelEncoder()
    y_train = le.fit_transform(df_train_bal['label'])
    X_train = df_train_bal[features]

    # --- 6. HUẤN LUYỆN LIGHTGBM ---
    print("🚀 Đang huấn luyện LightGBM...")
    params = {
        'objective': 'multiclass',
        'num_class': len(le.classes_),
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'random_state': 42,
        'learning_rate': 0.05,
        'n_estimators': 500,
        'verbosity': -1
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    print("🎉 Huấn luyện xong!")

else:
    print("❌ Không load được dữ liệu nào. Kiểm tra lại đường dẫn hoặc dataset.")

# --- 7. HÀM POST-PROCESSING ---
def run_length_encoding_pro(predictions, agent_id, target_id):
    events = []
    if len(predictions) == 0: return events
    
    current_label = predictions[0]
    start_frame = 0
    
    for i in range(1, len(predictions)):
        if predictions[i] != current_label:
            if current_label != 'other':
                events.append({
                    'agent_id': agent_id,
                    'target_id': target_id,
                    'action': current_label,
                    'start_frame': start_frame,
                    'stop_frame': i - 1
                })
            current_label = predictions[i]
            start_frame = i
            
    if current_label != 'other':
        events.append({
            'agent_id': agent_id,
            'target_id': target_id,
            'action': current_label,
            'start_frame': start_frame,
            'stop_frame': len(predictions) - 1
        })
    return events

# --- 8. TẠO SUBMISSION (SAFE MODE) ---
if 'model' in locals(): # Chỉ chạy nếu train thành công
    print("📝 Đang tạo submission...")
    try:
        df_test_meta = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
    except: df_test_meta = pd.DataFrame()

    submission_rows = []
    row_id_counter = 0

    for idx, row in tqdm(df_test_meta.iterrows(), total=len(df_test_meta)):
        try:
            video_id = row['video_id']
            lab_id = row['lab_id']
            pix_per_cm = row['pix_per_cm_approx'] if row['pix_per_cm_approx'] > 0 else 1.0
            
            t_path = os.path.join(DATA_PATH, 'test_tracking', lab_id, f'{video_id}.parquet')
            if not os.path.exists(t_path): continue
            
            df_track = pd.read_parquet(t_path)
            px = df_track.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='x')
            px.columns = [f"mouse{m}_{bp}_x" for m, bp in px.columns]
            py = df_track.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='y')
            py.columns = [f"mouse{m}_{bp}_y" for m, bp in py.columns]
            df_wide = pd.concat([px, py], axis=1).sort_index(axis=1)
            
            df_wide = df_wide.interpolate(limit=5).fillna(0)
            df_wide = df_wide / pix_per_cm
            
            mouse_ids = sorted(list(set([int(c.split('_')[0].replace('mouse', '')) for c in df_wide.columns if 'mouse' in c])))
            
            for m1, m2 in itertools.permutations(mouse_ids, 2):
                cols1 = [c for c in df_wide.columns if f'mouse{m1}_' in c]
                cols2 = [c for c in df_wide.columns if f'mouse{m2}_' in c]
                if not cols1 or not cols2: continue
                
                df_pair = df_wide[cols1 + cols2].copy()
                rename_dict = {}
                for c in cols1: rename_dict[c] = c.replace(f'mouse{m1}_', 'mouse1_')
                for c in cols2: rename_dict[c] = c.replace(f'mouse{m2}_', 'mouse2_')
                df_pair.rename(columns=rename_dict, inplace=True)
                
                # Dùng hàm SAFE features
                df_pair = calculate_features_safe(df_pair)
                
                X_test = pd.DataFrame(0.0, index=df_pair.index, columns=features)
                for c in features:
                    if c in df_pair.columns: X_test[c] = df_pair[c]
                
                # Dự đoán
                y_pred_idx = model.predict(X_test)
                y_pred_lbl = le.inverse_transform(y_pred_idx)
                
                events = run_length_encoding_pro(y_pred_lbl, m1, m2)
                
                for event in events:
                    submission_rows.append({
                        'row_id': row_id_counter,
                        'video_id': video_id,
                        'agent_id': event['agent_id'],
                        'target_id': event['target_id'],
                        'action': event['action'],
                        'start_frame': event['start_frame'],
                        'stop_frame': event['stop_frame']
                    })
                    row_id_counter += 1
                    
            del df_wide
            gc.collect()
        except Exception as e:
            print(f"Error processing test video {video_id}: {e}")
            continue

    # Lưu file
    sub = pd.DataFrame(submission_rows)
    if len(sub) == 0:
        sub = pd.DataFrame(columns=['row_id', 'video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])

    sub.to_csv('submission.csv', index=False)
    print(f"✅ Đã tạo submission.csv với {len(sub)} dòng.")
    display(sub.head())

⏳ Đang tạo dữ liệu 'All-Pairs' từ 50 video (Safe Mode)...


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Dữ liệu thô: (8829739, 7)
⚖️ Đang cân bằng dữ liệu...
✅ Dữ liệu train cuối cùng: (418352, 7)
🚀 Đang huấn luyện LightGBM...
🎉 Huấn luyện xong!
📝 Đang tạo submission...


  0%|          | 0/1 [00:00<?, ?it/s]

✅ Đã tạo submission.csv với 6285 dòng.


,row_id,video_id,agent_id,target_id,action,start_frame,stop_frame
0,0,438887472,1,2,attack,0,0
1,1,438887472,1,2,approach,133,133
2,2,438887472,1,2,avoid,138,138
3,3,438887472,1,2,avoid,140,150
4,4,438887472,1,2,avoid,152,152
